# DHW Extraction

### Load your libraries

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
import os
import csv
from PIL import Image
import pandas as pd
from shapely.geometry import Point
from shapely.ops import unary_union
from geopandas import GeoDataFrame
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import numpy as np

### Define the functions

In [ ]:
def calculate_mean_dhw_value(point, dhw_folder, image_cache):
    dhw_values = []
    for filename, image_array in image_cache[dhw_folder].items():
        pixel_coords = point_to_pixel_coords(point, image_array)
        dhw_value = image_array[pixel_coords[1], pixel_coords[0]]
        dhw_values.append(dhw_value)
    mean_dhw = sum(dhw_values) / len(dhw_values)
    return mean_dhw

In [ ]:
def point_to_pixel_coords(point, image):
    img_width, img_height = image.size
    pixel_x = int((point.x / 360 + 0.5) * img_width)
    pixel_y = int((0.5 - point.y / 180) * img_height)
    return pixel_x, pixel_y  # Return as a tuple

In [ ]:
def load_images_into_cache(dhw_folder):
    image_cache = {}
    for filename in os.listdir(dhw_folder):
        if filename.endswith('.md5'):
            continue  # Skip MD5 checksum files
        file_path = os.path.join(dhw_folder, filename)
        image = Image.open(file_path)
        image_array = np.array(image)
        image_cache[filename] = image_array
    return image_cache


### Read the CSV file containing point information:

In [ ]:
point_data = pd.read_csv('../Data/site_coord_geoenrich2_LM.csv')
point_data['date'] = pd.to_datetime(point_data['date'])  # Convert 'date' column to datetime

point_data = point_data[:2]
print(point_data)

###     Iterate over each row in the CSV file to calculate the mean DHW value for each point:

In [ ]:
dhw_results = []
num_points = len(point_data)

with tqdm(total=num_points, desc="Calculating Mean DHW") as pbar:
    image_cache = {}  # Define image_cache variable
    
    # Load images into cache for the year 2009
    dhw_folder_2009 = "../DHW/2009"
    image_cache[dhw_folder_2009] = load_images_into_cache(dhw_folder_2009)


    def process_point(row, image_cache):
        point_id = row['id']
        latitude = row['latitude']
        longitude = row['longitude']
        date = row['date']

        point = Point(longitude, latitude)
        buffer_radius = 10000  # 10 km buffer

        buffer = point.buffer(buffer_radius)
        buffer_gdf = GeoDataFrame(geometry=[buffer])

        year_folder = str(date.year)
        dhw_folder = os.path.join('../DHW', year_folder)

        mean_dhw = calculate_mean_dhw_value(point, dhw_folder, image_cache)
        return (point_id, mean_dhw)



    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_point, row, image_cache) for _, row in point_data.iterrows()]
        for future in tqdm(as_completed(futures), total=num_points):
            dhw_results.append(future.result())

### Create a new CSV file with the point IDs and the corresponding mean DHW values

In [ ]:
output_file = '../Results/dhw_all_1_mean.csv'
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['point_id', 'mean_dhw'])
    writer.writerows(dhw_results)
